In [ ]:
import numpy as np
import cv2

# Load the NPZ file
train_data = np.load(r'D:\Project\AFB\carto_mix_256_final.npz')
x_train = train_data['arr_0']  # Assuming 'images' is the key for image data
y_train = train_data['arr_1']  # Assuming 'labels' is the key for label data
print(x_train.shape)
# Convert images from RGB to Grayscale
gray_images_x_train = []
for img in x_train:
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray_images_x_train.append(gray_img)

# Convert the list to a numpy array
gray_images_x_train = np.array(gray_images_x_train)

# Save the grayscale images and labels back into a new NPZ file
#np.savez('new_one_carto_mix_256_final.npz', images=gray_images, labels=labels)
print(gray_images_x_train.shape)

In [ ]:
test_data = np.load(r'D:\Project\AFB\carto_mix_256_test.npz')
x_test = test_data['arr_0']
y_test = test_data['arr_1']

gray_images_x_test = []
for img in x_test:
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray_images_x_test.append(gray_img)

# Convert the list to a numpy array
gray_images_x_test = np.array(gray_images_x_test)
print(gray_images_x_test.shape)

In [ ]:
import numpy as np
import imgaug.augmenters as iaa

In [ ]:
vflip = iaa.Flipud(1.0)
hflip = iaa.Fliplr(1.0)
rotate90 = iaa.Affine(rotate=90)
rotate180 = iaa.Affine(rotate=180)
rotate270 = iaa.Affine(rotate=270)
rotate360 = iaa.Affine(rotate=360)
transpose = iaa.Sequential([iaa.Affine(rotate=90), iaa.Flipud(1.0)])
shift_scale_rotate = iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, scale=(0.8, 1.2), rotate=(-45, 45))

# Lists to store augmented images and labels
vflip_images, vflip_labels = [], []
hflip_images, hflip_labels = [], []
tp_images, tp_labels = [], []
ssr_images, ssr_labels = [], []
t90_images, t90_labels = [], []
t180_images, t180_labels = [], []
t270_images, t270_labels = [], []
t360_images, t360_labels = [], []

# Function to apply augmentation
def augment_data(augmenter, images, labels):
    aug_images, aug_labels = [], []
    for img, lbl in zip(images, labels):
        aug_img = augmenter(image=img)
        aug_lbl = augmenter(image=lbl)
        aug_images.append(aug_img)
        aug_labels.append(aug_lbl)
    return aug_images, aug_labels

# Apply augmentations
vflip_images, vflip_labels = augment_data(vflip, gray_images_x_train, y_train)
hflip_images, hflip_labels = augment_data(hflip,gray_images_x_train, y_train)
tp_images, tp_labels = augment_data(transpose, gray_images_x_train, y_train)
ssr_images, ssr_labels = augment_data(shift_scale_rotate, gray_images_x_train, y_train)
t90_images, t90_labels = augment_data(rotate90, gray_images_x_train, y_train)
t180_images, t180_labels = augment_data(rotate180, gray_images_x_train, y_train)
t270_images, t270_labels = augment_data(rotate270, gray_images_x_train, y_train)
t360_images, t360_labels = augment_data(rotate360, gray_images_x_train, y_train)

# Concatenate the original and augmented data
X_train_final = np.array(list(gray_images_x_train) + vflip_images + hflip_images + tp_images + ssr_images + t90_images + t180_images + t270_images + t360_images)
y_train_final = np.array(list(y_train) + vflip_labels + hflip_labels + tp_labels + ssr_labels + t90_labels + t180_labels + t270_labels + t360_labels)

In [ ]:
print(X_train_final.shape)
X_train = X_train_final[..., np.newaxis]
print(y_train_final.shape)
print(X_train.shape)

In [ ]:
import keras
from keras.callbacks import ModelCheckpoint
from segmentation_models import Unet, Linknet, PSPNet, FPN
import segmentation_models as sm
from keras.utils import plot_model
import numpy as np
from numpy import savez_compressed,savez
from numpy import load
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score

In [ ]:
input_shape = (256, 256, 3)
model1 = sm.Linknet('resnet34', input_shape=input_shape,weights=None, classes=1, activation='sigmoid')

# Define the optimizer
opt = keras.optimizers.Adam(learning_rate=0.0001)

# Compile the model with the desired loss function and metrics
model1.compile(optimizer=opt, loss=sm.losses.bce_jaccard_loss, metrics=[sm.metrics.iou_score])
model1.summary()

In [ ]:
X_train_rgb = np.repeat(X_train, 3, axis=-1)

In [ ]:
print(X_train_rgb.shape)
print(y_train_final.shape)

In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    filepath='Linknetweights.{epoch:02d}-{val_loss:.2f}.h5',
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [ ]:
# Fit the model to your training data
model1.fit(
    X_train_rgb, y_train_final,
    batch_size=16,
    epochs=50,
    validation_split=0.2,
    callbacks=[model_checkpoint_callback]
)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from lime import lime_image
from skimage.segmentation import mark_boundaries
from tensorflow.keras.models import load_model

model_path = r'D:\Project\new_Linknetweights.01-1.39.h5'
model = load_model(model_path, compile=False)

ValueError: Unknown layer: Functional

In [ ]:
import h5py

model_path = r'D:\Project\new_Linknetweights.01-1.39.h5'
with h5py.File(model_path, 'r') as f:
    print(list(f.keys()))
    if 'model_config' in f.attrs:
        print("Model config found.")
    else:
        print("Model config not found.")

In [ ]:
# Function to preprocess images (if necessary)
def preprocess_input(img):
    # Convert RGB image to grayscale
    gray_img = rgb2gray(img)
    # Create fake RGB image by repeating grayscale values across three channels
    fake_rgb_img = np.repeat(gray_img[..., np.newaxis], 3, axis=-1)
    # Normalize or preprocess your image data as required
    return fake_rgb_img / 255.0

# Load and preprocess a sample RGB image
sample_rgb_image = X_train[0]  # Replace with your RGB image data
processed_image = preprocess_input(sample_rgb_image)

# Create a LIME image explainer
explainer = lime_image.LimeImageExplainer()

# Define a function for LIME to use for predictions
def predict_fn(images):
    # Ensure input matches expected shape (adjust as per your model's requirements)
    predictions = model.predict(images[np.newaxis, ...])
    # Example: Extract probability map for foreground class (class 1)
    return predictions[..., 1]  # Assuming class 1 is foreground

# Explain the prediction on the sample image using LIME
explanation = explainer.explain_instance(
    image=processed_image.astype(np.uint8),  # Ensure image is in correct format for LIME
    classifier_fn=predict_fn,
    top_labels=1,
    hide_color=0,
    num_samples=1000  # Number of perturbed samples to generate
)

# Get the explanation for the top label (adjust as per your model's output)
temp, mask = explanation.get_image_and_mask(
    label=explanation.top_labels[0],
    positive_only=True,
    num_features=10,
    hide_rest=False
)

In [ ]:
# Display the result
plt.imshow(mark_boundaries(sample_rgb_image.astype(np.uint8), mask))
plt.title('LIME Explanation on Fake RGB Image')
plt.show()